

# Import libraries.


In [ ]:
# Core
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns;
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense,BatchNormalization,Dropout,Input
from keras.models import Sequential, Model
from keras.layers import Conv2D
from tensorflow.keras import datasets, layers, models
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from sklearn.metrics import plot_confusion_matrix ,classification_report
from sklearn.metrics import accuracy_score, precision_score,recall_score,f1_score


# bussiness task
The original MNIST image dataset of handwritten digits is a popular benchmark for image-based machine learning methods but researchers have renewed efforts to update it and develop drop-in replacements that are more challenging for computer vision and original for real-world applications. As noted in one recent replacement called the Fashion-MNIST dataset, the Zalando researchers quoted the startling claim that "Most pairs of MNIST digits (784 total pixels per sample) can be distinguished pretty well by just one pixel". To stimulate the community to develop more drop-in replacements, the Sign Language MNIST is presented here and follows the same CSV format with labels and pixel values in single rows. The American Sign Language letter database of hand gestures represent a multi-class problem with 24 classes of letters (excluding J and Z which require motion).

The dataset format is patterned to match closely with the classic MNIST. Each training and test case represents a label (0-25) as a one-to-one map for each alphabetic letter A-Z (and no cases for 9=J or 25=Z because of gesture motions). The training data (27,455 cases) and test data (7172 cases) are approximately half the size of the standard MNIST but otherwise similar with a header row of label, pixel1,pixel2….pixel784 which represent a single 28x28 pixel image with grayscale values between 0-255. The original hand gesture image data represented multiple users repeating the gesture against different backgrounds. The Sign Language MNIST data came from greatly extending the small number (1704) of the color images included as not cropped around the hand region of interest. To create new data, an image pipeline was used based on ImageMagick and included cropping to hands-only, gray-scaling, resizing, and then creating at least 50+ variations to enlarge the quantity. The modification and expansion strategy was filters ('Mitchell', 'Robidoux', 'Catrom', 'Spline', 'Hermite'), along with 5% random pixelation, +/- 15% brightness/contrast, and finally 3 degrees rotation. Because of the tiny size of the images, these modifications effectively alter the resolution and class separation in interesting, controllable ways.

This dataset was inspired by the Fashion-MNIST 2 and the machine learning pipeline for gestures by Sreehari 4.

A robust visual recognition algorithm could provide not only new benchmarks that challenge modern machine learning methods such as Convolutional Neural Nets but also could pragmatically help the deaf and hard-of-hearing better communicate using computer vision applications. The National Institute on Deafness and other Communications Disorders (NIDCD) indicates that the 200-year-old American Sign Language is a complete, complex language (of which letter gestures are only part) but is the primary language for many deaf North Americans. ASL is the leading minority language in the U.S. after the "big four": Spanish, Italian, German, and French. One could implement computer vision in an inexpensive board computer like Raspberry Pi with OpenCV, and some Text-to-Speech to enabling improved and automated translation applications


In [ ]:
seed = 42
np.random.seed =seed

# Reading Data

In [ ]:
train_path = '../input/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv'
test_path  = '../input/sign-language-mnist/sign_mnist_test/sign_mnist_test.csv'

In [ ]:
train = pd.read_csv(train_path)
test  = pd.read_csv(test_path)

In [ ]:
train.head(10)

In [ ]:
test.head(10)

# Data Cleaning
1.   Check Null data 
2.   Drop Non Important Data
3.   check douplicate data 


In [ ]:
cols =train.columns
colours = ['#747BA1', '#FFBA77'] # specify the orange  - yellow is missing. blue is not missing.
sns.heatmap(train[cols].isnull(), cmap=sns.color_palette(colours))

In [ ]:
cols =test.columns
colours = ['#747BA1', '#FFBA77'] # specify the orange  - yellow is missing. blue is not missing.
sns.heatmap(test[cols].isnull(), cmap=sns.color_palette(colours))

In [ ]:
train.isna().sum().sort_values(ascending=True)

In [ ]:
test.isna().sum().sort_values(ascending=True)

In [ ]:
train = train.drop_duplicates()
test  = test.drop_duplicates()

In [ ]:
# here in this part we will ckeck shape of data so we want to reshape data to be (x,28,28,1)

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
label = train['label']
train = train.drop(['label'],axis=1)

In [ ]:
test_label = test['label']
test = test.drop(['label'],axis=1)

In [ ]:
print(train.shape)
print(test.shape)
print(label.shape)
print(test_label.shape)

# Data Visualization
in this part we will analays and versialize each part of data to be in near step from our goal then pased on deployed models we will sense best factior that affect on our bussiness goal



In [ ]:
sns.countplot(x= label)
plt.show()

# Data Normalization
hear we need to normalize data between 0,1 instead of 0,255 which nmuch better to make sure that all data have same importants

In [ ]:
train = train /255
test  = test /255

In [ ]:
train = np.array(train)
test  = np.array(test)

In [ ]:
train = train.reshape(train.shape[0],28,28,1)
test  = test.reshape(test.shape[0],28,28,1)

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
image_x = random.randint(0,len(test))
plt.imshow(np.reshape(train[image_x],(28,28)))
plt.title('Train Image')
plt.show()
plt.imshow(np.reshape(test[image_x],(28,28)))
plt.title('Test Image')
plt.show()


# Buliding Model

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(64,(3,3),padding ='Same',activation = 'relu',input_shape=(28,28,1)))
model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D(2,2))
model.add(Dropout(0.3))
model.add(layers.Conv2D(128,(3,3),padding ='same',activation='relu'))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D(2,2))
model.add(Dropout(0.3))
model.add(layers.Conv2D(128,(3,3),padding ='same',activation='relu'))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D(2,2))


In [ ]:
model.summary()

In [ ]:
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(layers.Dense(25, activation ='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])


In [ ]:
history = model.fit(train,label,epochs=10,validation_data=(test,test_label))


In [ ]:
model.evaluate(test, test_label)

# predection

In [ ]:
#get predection
predictions = model.predict(test)
predictions = np.argmax(predictions, axis = 1)
predictions[5]

In [ ]:
plt.imshow(np.reshape(test[5],(28,28)))
plt.title('Test Image')
plt.show()

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test,  test_label, verbose=2)